### Author

Recep Fırat Çekinel

In [1]:
from cifar10_models import resnet
import torch
import numpy as np
import pandas as pd
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
from time import process_time, process_time_ns

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
import wandb

wandb.login()
#!wandb login --relogin

wandb: Currently logged in as: firatc (use `wandb login --relogin` to force relogin)


True

In [4]:
sweep_config = {'method': 'grid'}

parameters_dict = {
    'learning_rate': {
        'values': [0.001, 0.01 ]
        },
    }

sweep_config['parameters'] = parameters_dict


sweep_id = wandb.sweep(sweep_config, project="mmi712-term-project")

Create sweep with ID: dvsgvufx
Sweep URL: https://wandb.ai/firatc/mmi712-term-project/sweeps/dvsgvufx


# Loading the Cifar-10 Dataset

In [3]:
# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='data/',
                                             train=True, 
                                             transform=transform,
                                             download=False)

test_dataset = torchvision.datasets.CIFAR10(root='data/',
                                            train=False, 
                                            transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100, 
                                          shuffle=False)

# Save & Load the Model

In [4]:
def saveModel(model, model_name):
    # Save the model checkpoint
    torch.save(model, model_name)

In [5]:
def loadModel(model_name, dev="cuda"):
    if dev == "cuda" and not torch.cuda.is_available():
        dev = "cpu"
    return torch.load(model_name, map_location=torch.device(dev))

# Test the Model on Eval Dataset

In [6]:
def testModel(model, test_loader):
    # Test the model
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

In [7]:
def getInference(instance, model, repetitions=300):
    
    #starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
    timings=np.zeros((repetitions,1))
    #GPU-WARM-UP
    # MEASURE PERFORMANCE
    model.eval()
    with torch.no_grad():
        instance = instance.to("cpu")
        for rep in range(repetitions):
            #starter.record()
            t_start = process_time_ns() 
            _ = model(instance)
            t_stop = process_time_ns() 
            #ender.record()
            # WAIT FOR GPU SYNC
            #torch.cuda.synchronize()
            #curr_time = starter.elapsed_time(ender)
            curr_time = (t_stop - t_start) / 10 **9
            timings[rep] = curr_time
    mean_syn = np.sum(timings) / repetitions
    std_syn = np.std(timings)
    
    return (mean_syn, std_syn)

In [8]:
# take a sample data instance to measure the inference
test_loader2 = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=1, 
                                          shuffle=False)
for sample_image, sample_label in test_loader2:  
    print(sample_image, sample_label)
    break

tensor([[[[0.6196, 0.6235, 0.6471,  ..., 0.5373, 0.4941, 0.4549],
          [0.5961, 0.5922, 0.6235,  ..., 0.5333, 0.4902, 0.4667],
          [0.5922, 0.5922, 0.6196,  ..., 0.5451, 0.5098, 0.4706],
          ...,
          [0.2667, 0.1647, 0.1216,  ..., 0.1490, 0.0510, 0.1569],
          [0.2392, 0.1922, 0.1373,  ..., 0.1020, 0.1137, 0.0784],
          [0.2118, 0.2196, 0.1765,  ..., 0.0941, 0.1333, 0.0824]],

         [[0.4392, 0.4353, 0.4549,  ..., 0.3725, 0.3569, 0.3333],
          [0.4392, 0.4314, 0.4471,  ..., 0.3725, 0.3569, 0.3451],
          [0.4314, 0.4275, 0.4353,  ..., 0.3843, 0.3725, 0.3490],
          ...,
          [0.4863, 0.3922, 0.3451,  ..., 0.3804, 0.2510, 0.3333],
          [0.4549, 0.4000, 0.3333,  ..., 0.3216, 0.3216, 0.2510],
          [0.4196, 0.4118, 0.3490,  ..., 0.3020, 0.3294, 0.2627]],

         [[0.1922, 0.1843, 0.2000,  ..., 0.1412, 0.1412, 0.1294],
          [0.2000, 0.1569, 0.1765,  ..., 0.1216, 0.1255, 0.1333],
          [0.1843, 0.1294, 0.1412,  ..., 0

In [10]:
model = loadModel("resnet_lr0.01.pth")
testModel(model, test_loader)
model = model.to("cpu")
getInference(sample_image, model, 100)

Accuracy of the model on the test images: 90.43 %


(0.04692480155, 0.006379768579961289)

# Training a Resnet Model

In [9]:
def trainModel(model, learning_rate, train_loader):
    # Hyper-parameters
    num_epochs = 80
    #learning_rate = 0.001
    
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # For updating learning rate
    def update_lr(optimizer, lr):    
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

    # Train the model
    total_step = len(train_loader)
    curr_lr = learning_rate
    for epoch in tqdm(range(num_epochs)):
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i+1) % 100 == 0:
                print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}".format(epoch+1, num_epochs, i+1, total_step, loss.item()))
                wandb.log({"Epoch": epoch+1, "Loss": loss.item()})

        # Decay learning rate
        if (epoch+1) % 20 == 0:
            curr_lr /= 3
            update_lr(optimizer, curr_lr)
    
    # Save the model checkpoint
    #torch.save(model, 'resnet_lr'+str(learning_rate)+'.pth')
    saveModel(model, 'resnet_lr'+str(learning_rate)+'.pth')
    testModel(model, test_loader)

In [10]:
def init_model(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        model = resnet.resnet18().to(device)
        
        trainModel(model, config.learning_rate, train_loader)

In [11]:
wandb.agent(sweep_id, init_model)

wandb: Agent Starting Run: ef7yabdt with config:
wandb: 	learning_rate: 0.001
wandb: wandb version 0.12.9 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


  0%|          | 0/80 [00:00<?, ?it/s]

Epoch [1/80], Step [100/500] Loss: 1.6570
Epoch [1/80], Step [200/500] Loss: 1.4522
Epoch [1/80], Step [300/500] Loss: 1.2114
Epoch [1/80], Step [400/500] Loss: 1.1740


  1%|▏         | 1/80 [00:18<24:03, 18.27s/it]

Epoch [1/80], Step [500/500] Loss: 1.1244
Epoch [2/80], Step [100/500] Loss: 0.9098
Epoch [2/80], Step [200/500] Loss: 0.8929
Epoch [2/80], Step [300/500] Loss: 1.0156
Epoch [2/80], Step [400/500] Loss: 0.9678


...

Epoch [80/80], Step [500/500] Loss: 0.0133


Accuracy of the model on the test images: 90.43 %


Epoch,80
Loss,0.01325
_runtime,1435
_timestamp,1642075502
_step,399


Epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
Loss,█▅▄▃▄▄▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


# Pruning

In [15]:
model.modules

<bound method Module.modules of ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   

In [16]:
prune_modules = []
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Conv2d):
        print(name, module)
        if "downsample" not in name:
            prune_modules.append(name)
    # prune 40% of connections in all linear layers
    elif isinstance(module, torch.nn.Linear):
        print(name, module)
        prune_modules.append(name)

conv1 Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
layer1.0.conv1 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
layer1.0.conv2 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
layer1.1.conv1 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
layer1.1.conv2 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
layer2.0.conv1 Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
layer2.0.conv2 Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
layer2.0.downsample.0 Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
layer2.1.conv1 Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
layer2.1.conv2 Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
layer3.0.conv1 Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)

In [9]:
def getParams(model):
    parameters_to_prune = (
        (model.conv1, 'weight'),
        (model.layer1[0].conv1, 'weight'),
        (model.layer1[0].conv2, 'weight'),
        (model.layer1[1].conv1, 'weight'),
        (model.layer1[1].conv2, 'weight'),
        (model.layer2[0].conv1, 'weight'),
        (model.layer2[0].conv2, 'weight'),
        (model.layer2[1].conv1, 'weight'),
        (model.layer2[1].conv2, 'weight'),
        (model.layer3[0].conv1, 'weight'),
        (model.layer3[0].conv2, 'weight'),
        (model.layer3[1].conv1, 'weight'),
        (model.layer3[1].conv2, 'weight'),
        (model.layer4[0].conv1, 'weight'),
        (model.layer4[0].conv2, 'weight'),
        (model.layer4[1].conv1, 'weight'),
        (model.layer4[1].conv2, 'weight'),
        (model.fc, 'weight'),
    )
    return parameters_to_prune

In [10]:
def pruneModel(model, parameters_to_prune, ratio, printStats=True):
    prune.global_unstructured(
        parameters_to_prune,
        pruning_method=prune.L1Unstructured,
        amount=ratio,
    )
    
    if not printStats:
        return
    
    print(
        "Sparsity in conv1.weight: {:.2f}%".format(
            100. * float(torch.sum(model.conv1.weight == 0))
            / float(model.conv1.weight.nelement())
        )
    )
    print(
        "Sparsity in layer1[0].conv1.weight: {:.2f}%".format(
            100. * float(torch.sum(model.layer1[0].conv1.weight == 0))
            / float(model.layer1[0].conv1.weight.nelement())
        )
    )
    print(
        "Sparsity in layer1[0].conv2.weight: {:.2f}%".format(
            100. * float(torch.sum(model.layer1[0].conv2.weight == 0))
            / float(model.layer1[0].conv2.weight.nelement())
        )
    )
    print(
        "Sparsity in layer1[1].conv1.weight: {:.2f}%".format(
            100. * float(torch.sum(model.layer1[1].conv1.weight == 0))
            / float(model.layer1[1].conv1.weight.nelement())
        )
    )
    print(
        "Sparsity in layer1[1].conv2.weight: {:.2f}%".format(
            100. * float(torch.sum(model.layer1[1].conv2.weight == 0))
            / float(model.layer1[1].conv2.weight.nelement())
        )
    )
    print(
        "Sparsity in layer2[0].conv1.weight: {:.2f}%".format(
            100. * float(torch.sum(model.layer2[0].conv1.weight == 0))
            / float(model.layer2[0].conv1.weight.nelement())
        )
    )
    print(
        "Sparsity in layer2[0].conv2.weight: {:.2f}%".format(
            100. * float(torch.sum(model.layer2[0].conv2.weight == 0))
            / float(model.layer2[0].conv2.weight.nelement())
        )
    )
    print(
        "Sparsity in layer2[1].conv1.weight: {:.2f}%".format(
            100. * float(torch.sum(model.layer2[1].conv1.weight == 0))
            / float(model.layer2[1].conv1.weight.nelement())
        )
    )
    print(
        "Sparsity in layer2[1].conv2.weight: {:.2f}%".format(
            100. * float(torch.sum(model.layer2[1].conv2.weight == 0))
            / float(model.layer2[1].conv2.weight.nelement())
        )
    )
    print(
        "Sparsity in layer3[0].conv1.weight: {:.2f}%".format(
            100. * float(torch.sum(model.layer3[0].conv1.weight == 0))
            / float(model.layer3[0].conv1.weight.nelement())
        )
    )
    print(
        "Sparsity in layer3[0].conv2.weight: {:.2f}%".format(
            100. * float(torch.sum(model.layer3[0].conv2.weight == 0))
            / float(model.layer3[0].conv2.weight.nelement())
        )
    )
    print(
        "Sparsity in layer3[1].conv1.weight: {:.2f}%".format(
            100. * float(torch.sum(model.layer3[1].conv1.weight == 0))
            / float(model.layer3[1].conv1.weight.nelement())
        )
    )
    print(
        "Sparsity in layer3[1].conv2.weight: {:.2f}%".format(
            100. * float(torch.sum(model.layer3[1].conv2.weight == 0))
            / float(model.layer3[1].conv2.weight.nelement())
        )
    )
    print(
        "Sparsity in layer4[0].conv1.weight: {:.2f}%".format(
            100. * float(torch.sum(model.layer4[0].conv1.weight == 0))
            / float(model.layer4[0].conv1.weight.nelement())
        )
    )
    print(
        "Sparsity in layer4[0].conv2.weight: {:.2f}%".format(
            100. * float(torch.sum(model.layer4[0].conv2.weight == 0))
            / float(model.layer4[0].conv2.weight.nelement())
        )
    )
    print(
        "Sparsity in layer4[1].conv1.weight: {:.2f}%".format(
            100. * float(torch.sum(model.layer4[1].conv1.weight == 0))
            / float(model.layer4[1].conv1.weight.nelement())
        )
    )
    print(
        "Sparsity in layer4[1].conv2.weight: {:.2f}%".format(
            100. * float(torch.sum(model.layer4[1].conv2.weight == 0))
            / float(model.layer4[1].conv2.weight.nelement())
        )
    )
    print(
        "Sparsity in fc1.weight: {:.2f}%".format(
            100. * float(torch.sum(model.fc.weight == 0))
            / float(model.fc.weight.nelement())
        )
    )

    print(
        "Global sparsity: {:.2f}%".format(
            100. * float(
                torch.sum(model.conv1.weight == 0)
                + torch.sum(model.layer1[0].conv1.weight == 0)
                + torch.sum(model.layer1[0].conv2.weight == 0)
                + torch.sum(model.layer1[1].conv1.weight == 0)
                + torch.sum(model.layer1[1].conv2.weight == 0)
                + torch.sum(model.layer2[0].conv1.weight == 0)
                + torch.sum(model.layer2[0].conv2.weight == 0)
                + torch.sum(model.layer2[1].conv1.weight == 0)
                + torch.sum(model.layer2[1].conv2.weight == 0)
                + torch.sum(model.layer3[0].conv1.weight == 0)
                + torch.sum(model.layer3[0].conv2.weight == 0)
                + torch.sum(model.layer3[1].conv1.weight == 0)
                + torch.sum(model.layer3[1].conv2.weight == 0)
                + torch.sum(model.layer4[0].conv1.weight == 0)
                + torch.sum(model.layer4[0].conv2.weight == 0)
                + torch.sum(model.layer4[1].conv1.weight == 0)
                + torch.sum(model.layer4[1].conv2.weight == 0)
                + torch.sum(model.fc.weight == 0)
            )
            / float(
                model.conv1.weight.nelement()
                + model.layer1[0].conv1.weight.nelement()
                + model.layer1[0].conv2.weight.nelement()
                + model.layer1[1].conv1.weight.nelement()
                + model.layer1[1].conv2.weight.nelement()
                + model.layer2[0].conv1.weight.nelement()
                + model.layer2[0].conv2.weight.nelement()
                + model.layer2[1].conv1.weight.nelement()
                + model.layer2[1].conv2.weight.nelement()
                + model.layer3[0].conv1.weight.nelement()
                + model.layer3[0].conv2.weight.nelement()
                + model.layer3[1].conv1.weight.nelement()
                + model.layer3[1].conv2.weight.nelement()
                + model.layer4[0].conv1.weight.nelement()
                + model.layer4[0].conv2.weight.nelement()
                + model.layer4[1].conv1.weight.nelement()
                + model.layer4[1].conv2.weight.nelement()
                + model.fc.weight.nelement()
            )
        )
    )

In [15]:
def setPruningPermanent(model, modules):
    for module, w in modules:
        prune.remove(module, 'weight')

# Re-evaluate the model after pruning

In [16]:
prune_ratios = np.arange(0.05, 0.85, 0.05)
for i in prune_ratios:
    print("Ratio: ", i)
    print("Model1: lr= ", 0.001)
    model = loadModel("resnet_lr0.001.pth")
    parameters_to_prune = getParams(model)
    pruneModel(model, parameters_to_prune, i, printStats=False)
    setPruningPermanent(model, parameters_to_prune)
    testModel(model, test_loader)
    model = model.to("cpu")
    mean, stdev = getInference(sample_image, model, 100)
    print("Inference: ", (mean, stdev))
    
    print("Model2: lr= ", 0.01)
    model = loadModel("resnet_lr0.01.pth")
    parameters_to_prune = getParams(model)
    pruneModel(model, parameters_to_prune, i, printStats=False)
    setPruningPermanent(model, parameters_to_prune)
    testModel(model, test_loader)
    model = model.to("cpu")
    mean, stdev = getInference(sample_image, model, 100)
    print("Inference: ", (mean, stdev))

Ratio:  0.05
Model1: lr=  0.001
Accuracy of the model on the test images: 90.84 %
Inference:  (0.04554151123000001, 0.007384506765080374)
Model2: lr=  0.01
Accuracy of the model on the test images: 90.44 %
Inference:  (0.045861150069999995, 0.005755680796214182)
Ratio:  0.1
Model1: lr=  0.001
Accuracy of the model on the test images: 90.78 %
Inference:  (0.04577782358999999, 0.0057789616452419874)
Model2: lr=  0.01
Accuracy of the model on the test images: 90.42 %
Inference:  (0.045754017550000005, 0.006526924194281888)
Ratio:  0.15000000000000002
Model1: lr=  0.001
Accuracy of the model on the test images: 90.73 %
Inference:  (0.05521959469000001, 0.011002556970250215)
Model2: lr=  0.01
Accuracy of the model on the test images: 90.42 %
Inference:  (0.04616189984999999, 0.006080496278343344)
Ratio:  0.2
Model1: lr=  0.001
Accuracy of the model on the test images: 90.03 %
Inference:  (0.045945243729999995, 0.0055283602325883014)
Model2: lr=  0.01
Accuracy of the model on the test images

In [20]:
model.layer1[0].conv1.weight

tensor([[[[-0.4249,  0.2550, -0.0000],
          [ 0.6017,  1.8117,  1.1083],
          [-0.0000,  1.4855,  1.5301]],

         [[-0.5792,  0.4112,  2.8071],
          [ 0.4502,  0.2385,  1.4300],
          [ 0.8258,  0.4430,  1.0861]],

         [[ 0.2909,  0.0000, -0.0000],
          [ 0.0000,  0.2227,  0.0000],
          [-0.0000,  0.0000, -0.1957]],

         ...,

         [[ 0.0000,  0.1557,  0.2102],
          [ 0.1737,  0.2955, -0.1892],
          [-0.2364,  0.0000, -0.1832]],

         [[-0.7010, -1.1919,  1.0778],
          [-0.1431, -1.3694, -0.9160],
          [ 0.3771, -0.1808, -0.6137]],

         [[-0.5214, -2.2948, -0.3137],
          [-0.4003, -5.5395, -2.8928],
          [ 0.7913, -1.9342, -0.7764]]],


        [[[ 0.6393,  0.3993,  0.9330],
          [-0.2474, -0.8675, -0.6761],
          [ 0.2971, -0.2367,  0.2297]],

         [[-0.9551, -0.4279,  0.0000],
          [-1.3315, -1.5427, -0.9220],
          [-0.3729, -1.0323, -0.4277]],

         [[ 0.4903,  0.2804,  0